In [2]:
!pip install sklearn --user

     |████████████████████████████████| 6.8MB 1.0MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=cec0b923b323ceea8d31be3ce7893c6eabb8b5da0728a7d3297ad4976b33d3e2
  Stored in directory: /home/s_r/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [1]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier 
import pickle 
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC,LinearSVC,NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/s_r/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/s_r/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
class VoteClassifier(ClassifierI):
    def __init__(self,*classifiers):
        self._classifiers = classifiers
        
    def classify(self,features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self,features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            
        choice_votes = votes.count(mode(votes))
        conf = choice_votes/len(votes)
        
        return conf

In [5]:
short_pos = [line.rstrip('\n') for line in open('short_reviews/positive.txt', 'r', encoding='ISO-8859-1')]
short_neg = [line.rstrip('\n') for line in open('short_reviews/negative.txt', 'r', encoding='ISO-8859-1')]

all_words = []
documents = []

# j is adjective, r is adverb , v is verb
# allowed_word_types = ["J","R","V"]
allowed_word_types = ["J","R","V"]

for p in short_pos:
    documents.append((p,"pos"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

for p in short_neg:
    documents.append((p,"neg"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

save_documents = open("pickled_algos/documents.pickle","wb")
pickle.dump(documents,save_documents)
save_documents.close()

all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:5000]

save_word_features = open("pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features,save_word_features)
save_word_features.close()

In [6]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
        
    return features

In [7]:
featuresets = [(find_features(rev),category) for (rev,category) in documents]

random.shuffle(featuresets)
print(len(featuresets))

testing_set = featuresets[5000:]
training_set = featuresets[:5000]

classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Naive Bayes Accuracy = {}".format(nltk.classify.accuracy(classifier,testing_set))*100)
classifier.show_most_informative_features(15)

save_classifier = open("pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier,save_classifier)
save_classifier.close()

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB Classifier Accuracy ={}".format(nltk.classify.accuracy(MNB_classifier,testing_set))*100)

save_classifier = open("pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier,save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("Bernoulli Classifier Accuracy = {}".format(nltk.classify.accuracy(BernoulliNB_classifier,testing_set))*100)

save_classifier = open("pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier,save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression Classifier Accuracy = {}".format(nltk.classify.accuracy(LogisticRegression_classifier,testing_set))*100)

save_classifier = open("pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier,save_classifier)
save_classifier.close()

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC Classifier Accuracy = {}".format(nltk.classify.accuracy(LinearSVC_classifier,testing_set))*100)

save_classifier = open("pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier,save_classifier)
save_classifier.close()

SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDC Classifier Accuracy = {}".format(nltk.classify.accuracy(SGDC_classifier,testing_set))*100)

save_classifier = open("pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier,save_classifier)
save_classifier.close()

voted_classifier = VoteClassifier(classifier,LinearSVC_classifier,MNB_classifier,BernoulliNB_classifier,LogisticRegression_classifier)
print("Voted classifier accuracy =",(nltk.classify.accuracy(voted_classifier,testing_set))*100)

7271
Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.802289740202554Naive Bayes Accuracy = 0.8022897402

Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707Bernoulli Classifier Accuracy = 0.7930427124614707

LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821LinearSVC Classifier Accuracy = 0.7683839718185821

Voted classifier accuracy = 79.56847203874945


In [ ]:
def sentiment(text):
    feats = find_features(text)
    
    return voted_classifier.classify(feats)